# Pytorch VGG-16 Fine tuning
Using PyTorch, I fine-tuned the learned weights for the VGG16 network architecture.
I'm sure there are a lot of things that could be improved, but I hope this will be helpful for everyone implementing this in PyTorch.
Please let me know if there's anything I should fix!

In [ ]:
import glob
import os.path as osp

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tqdm.notebook import tqdm
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
import cv2
import albumentations as A

In [ ]:
! ls /kaggle/input/resized-plant2021/

# Constant Config

In [ ]:
SELECT_MODEL = 'VGG16' # VGG16, AlexNet, DenseNet, submit
LOAD_PRETRAIN = True

PRETRAIN_PATH = '../input/finetuningmodelzoo/vgg16_final-50epoch_fine_tuning_v1.h' # VGG16
# PRETRAIN_PATH = '../input/finetuningmodelzoo/alexnet_final200epoch_fine_tuning_v1.h' # AlexNet
# PRETRAIN_PATH = '../input/finetuningmodelzoo/augment-dense-opensimpleclf-5.h' # DenseNet
SAVE_WEIGHT_FILENAME = 'augment-vgg16-opensimpleclf-1'

IS_SUBMIT = True

In [ ]:
CROP_WIDTH = 224
CROP_HEIGHT = 224 
SIZE = 256
MEAN = (0, 0, 0)
STD = (1, 1, 1)
if SELECT_MODEL == 'VGG16' or SELECT_MODEL == 'AlexNet':
    MEAN = (0.485, 0.456, 0.406)
    STD = (0.229, 0.224, 0.225)

AGU_PROB = 0.4

In [ ]:
TRAIN_IMAGE_PATH = '/kaggle/input/resized-plant2021/img_sz_256'
TEST_IMAGE_PATH = '../input/plant-pathology-2021-fgvc8/test_images'
# TEST_IMAGE_PATH = '../input/plant-pathology-2021-fgvc8/train_images'

# Data Process

## Import Data

In [ ]:
import pandas as pd
df_train = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/train.csv")
df_sub = pd.read_csv("/kaggle/input/plant-pathology-2021-fgvc8/sample_submission.csv")

* ### Check train

In [ ]:
display(df_train)
display(df_train.labels.value_counts())

In [ ]:
df_train.size

In [ ]:
d_set = set()
for k in df_train.labels.unique():
    d_set = d_set | set(k.split(" "))
print(f"num of labels: {len(d_set)}  {d_set}")

* ### Check submission

In [ ]:
display(df_sub)

## Label encoding

In [ ]:
def to_label(df):
    """
    Function for Label encoding.
    """
    le = LabelEncoder()
    df["labels_n"] = le.fit_transform(df.labels.values)
    return df

df_train = to_label(df_train)
df_labels_idx = df_train.loc[df_train.duplicated(["labels", "labels_n"])==False]\
                [["labels_n", "labels"]].set_index("labels_n").sort_index()
display(df_labels_idx)

## Image Path Builder

In [ ]:
def make_datapath_list(phase="train", val_size=0.25):
    """
    Function to create a PATH to the data.
    
    Parameters
    ----------
    phase : 'train' or 'val' or 'test'
        Specify whether to use Train data or test data.
    val_size : float
        Ratio of validation data to train data
        
    Returns
    -------
    path_lsit : list
        A list containing the PATH to the data.
    """
    
    if phase in ["train", "val"]:
        phase_path = "train_images"
    elif phase in ["test"]:
        phase_path = "test_images"
    else:
        print(f"{phase} not in path")
    rootpath = "/kaggle/input/plant-pathology-2021-fgvc8/"
#     rootpath = "/kaggle/input/resized-plant2021/img_sz_256/"
    target_path = osp.join(TRAIN_IMAGE_PATH , '*.jpg') if  phase in ['train', 'val'] else osp.join(TEST_IMAGE_PATH, "*.jpg")

    path_list = []
    
    for path in glob.glob(target_path):
        path_list.append(path)
        
    if phase in ["train", "val"]:
        train, val = train_test_split(path_list, test_size=val_size, random_state=0, shuffle=True)
        if phase == "train":
            path_list = train
        else:
            path_list = val
    
    return path_list

In [ ]:
train_list = make_datapath_list(phase="train")
print(f"train data length : {len(train_list)}")
val_list = make_datapath_list(phase="val")
print(f"validation data length : {len(val_list)}")
test_list = make_datapath_list(phase="test")
print(f"test data length : {len(test_list)}")

## Image Augmentation

* ## Bee Augmentation

In [ ]:
import random
import os
class InsectAugmentation:
    
    def __init__(self, p):
        self.p = p


    def __call__(self, image, n_insects=2, dark_insect=False, p=0.5, insects_folder='../input/bee-augmentation'):
        aug_prob = random.random()
        if aug_prob < self.p:
            height, width, _ = image.shape  # target image width and height
            insects_images = [im for im in os.listdir(insects_folder) if 'bee' in im]
            img_shape = image.shape

            for _ in range(n_insects):
                insect = cv2.cvtColor(cv2.imread(os.path.join(insects_folder, random.choice(insects_images))), cv2.COLOR_BGR2RGB)
                insect = cv2.flip(insect, random.choice([-1, 0, 1]))
                insect = cv2.rotate(insect, random.choice([0, 1, 2]))
                insect = cv2.resize(insect, (width, height))

                h_height, h_width, _ = insect.shape  # insect image width and height
                roi_ho = random.randint(0, image.shape[0] - insect.shape[0])
                roi_wo = random.randint(0, image.shape[1] - insect.shape[1])
                roi = image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width]

                # Creating a mask and inverse mask 
                img2gray = cv2.cvtColor(insect, cv2.COLOR_BGR2GRAY)
                ret, mask = cv2.threshold(img2gray, 10, 255, cv2.THRESH_BINARY)
                #mask_inv = cv2.cvtColor(cv2.bitwise_not(mask),cv2.COLOR_BGR2GRAY)
                mask_inv = cv2.bitwise_not(mask)

                # Now black-out the area of insect in ROI
                img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)

                # Take only region of insect from insect image.
                if dark_insect:
                    img_bg = cv2.bitwise_and(roi, roi, mask=mask_inv)
                    insect_fg = cv2.bitwise_and(img_bg, img_bg, mask=mask)
                else:
                    insect_fg = cv2.bitwise_and(insect, insect, mask=mask)

                # Put insect in ROI and modify the target image
                dst = cv2.add(img_bg, insect_fg, dtype=cv2.CV_64F)

                image[roi_ho:roi_ho + h_height, roi_wo:roi_wo + h_width] = dst

        return {'image': image}

* ## Basic Agumentation

In [ ]:
import albumentations as A
ALBUMENTATION_TRAIN_LIST = [
    A.SmallestMaxSize(SIZE, p=1),
    A.RandomCrop(p=1,height = CROP_HEIGHT, width = CROP_WIDTH),
#     A.RandomSunFlare(p=1),  #REMOVE
    A.RandomFog(p=AGU_PROB),
    A.RandomBrightness(p=AGU_PROB),
    A.Rotate(p=AGU_PROB, limit=90),
    A.RGBShift(p=AGU_PROB), 
#     A.RandomSnow(p=AGU_PROB), #REMOVE
    A.HorizontalFlip(p=AGU_PROB), 
    A.VerticalFlip(p=AGU_PROB), 
    A.RandomContrast(limit = 0.5,p = AGU_PROB),
    A.HueSaturationValue(p=AGU_PROB,hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=50),
    A.Cutout(p=AGU_PROB),
    A.Transpose(p=AGU_PROB), 
    A.JpegCompression(p=AGU_PROB),
    A.CoarseDropout(p=AGU_PROB),
    A.IAAAdditiveGaussianNoise(loc=0, scale=(2.5500000000000003, 12.75), per_channel=False, p=AGU_PROB),
    A.IAAAffine(scale=1.0, translate_percent=None, translate_px=None, rotate=0.0, shear=0.0, order=1, cval=0, mode='reflect', p=AGU_PROB),
    A.IAAAffine(rotate=90., p=AGU_PROB),
    A.IAAAffine(rotate=180., p=AGU_PROB),
    A.Normalize(MEAN, STD)
] if SELECT_MODEL == 'DenseNet' else [
    A.SmallestMaxSize(SIZE, p=1),
    A.RandomCrop(p=1,height = CROP_HEIGHT, width = CROP_WIDTH),
    A.HorizontalFlip(p=AGU_PROB),
    A.Normalize(MEAN, STD)
]

In [ ]:
ALBUMENTATION_VALID_LIST = [
    A.SmallestMaxSize(SIZE, p=1),
    A.CenterCrop(width = CROP_WIDTH, height = CROP_HEIGHT, p = 1),
    A.Normalize(MEAN, STD)
]

In [ ]:

# ALBUMENTATION_TEST_LIST = A.Compose([
#     A.SmallestMaxSize(SIZE, p=1),
#     A.CenterCrop(width = CROP_WIDTH, height = CROP_HEIGHT, p = 1),
#     A.Normalize(MEAN, STD)
# ])


if SELECT_MODEL == 'VGG16':
    ALBUMENTATION_TEST_LIST = transforms.Compose([
                    transforms.Resize(SIZE),
                    transforms.CenterCrop(SIZE),
                    transforms.ToTensor(),
                    transforms.Normalize(MEAN, STD)
                ]) 
    print('VGG16 - using torch vision for processing image')

* ## Image Transform Class

In [ ]:
class ImageTransform():
    """
    Class for image preprocessing.
    
    Attributes
    ----------
    resize : int
        224
    mean : (R, G, B)
        Average value for each color channel
    std : (R, G, B)
        Standard deviation for each color channel
    """
    

    
    def __init__(self):
        self.data_transform = {
            'train': A.Compose(ALBUMENTATION_TRAIN_LIST),
            'val': A.Compose(ALBUMENTATION_VALID_LIST),
            'test': ALBUMENTATION_TEST_LIST
        }
        
        self.bee_transform = InsectAugmentation(p = AGU_PROB)

    def __call__(self, img, phase="train"):
        """
        Parameters
        ----------
        phase: 'train' or 'val' or 'test'
            Specify the mode of preprocessing
        """
        if SELECT_MODEL == 'VGG16' and phase == 'test':
            agu_img = self.data_transform[phase](img)
            return agu_img
            
        agu_img = self.data_transform[phase](image = np.array(img))
#         agu_img = self.bee_transform(agu_img['image']) # BEE-AUGMENTATION
        norm_img =  torch.from_numpy(agu_img['image'].transpose(2,0,1))
        return norm_img

image_file_path = '/kaggle/input/resized-plant2021/img_sz_256/800113bb65efe69e.jpg'
img = Image.open(image_file_path)
# plt.imshow(img)
transform = ImageTransform()
img_transformed = transform(img, phase='train')
plt.imshow(img_transformed.numpy().transpose(1,2,0))

* ## Test Augmentation

+ ### Multiple Augment

In [ ]:
ROW = 10
COL = 10

def test_agument():
    image_file_path = '/kaggle/input/resized-plant2021/img_sz_256/800113bb65efe69e.jpg'
    img = Image.open(image_file_path)
    transform = ImageTransform()
    fig = plt.figure(figsize=(20, 20))
    for i in range(ROW * COL):
        fig.add_subplot(ROW, COL, i+1)
        img_transformed = transform(img, phase='train')
        plt.imshow(img_transformed.numpy().transpose(1,2,0))
        
test_agument()

## Build Dataset Object 

In [ ]:
class PlantDataset(data.Dataset):
    """
    Class to create a Dataset
    
    Attributes
    ----------
    df_train : DataFrame
        DataFrame containing the image labels.
    file_list : list
        A list containing the paths to the images
    transform : object
        Instance of the preprocessing class (ImageTransform)
    phase : 'train' or 'val' or 'test'
        Specify whether to use train, validation, or test
    """
    def __init__(self, df_train, file_list, transform=None, phase='train'):
        self.df_train = df_train
        self.df_labels_idx = df_labels_idx
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
   
    def __len__(self):
        """
        Returns the number of images.
        """
        return len(self.file_list)
    
    def __getitem__(self, index):
        """
        Get data in Tensor format and labels of preprocessed images.
        """
        #print(index)
        
        # Load the index number image.
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        # Preprocessing images
        img_transformed = self.transform(img, self.phase)
        
        # image name
        image_name = img_path[-20:]
        
        # Extract the labels
        if self.phase in ["train", "val"]:
            label = df_train.loc[df_train["image"]==image_name]["labels_n"].values[0]
        elif self.phase in ["test"]:
            label = -1
        
        return img_transformed, label, image_name

In [ ]:
train_dataset = PlantDataset(df_train, train_list, transform=ImageTransform(), phase='train')
val_dataset = PlantDataset(df_train, val_list, transform=ImageTransform(), phase='val')
test_dataset = PlantDataset(df_train, test_list, transform=ImageTransform(), phase='test')

index = 0

print("【train dataset】")
print(f"img num : {train_dataset.__len__()}")
print(f"img : {train_dataset.__getitem__(index)[0].size()}")
print(f"label : {train_dataset.__getitem__(index)[1]}")
print(f"image name : {train_dataset.__getitem__(index)[2]}")

print("\n【validation dataset】")
print(f"img num : {val_dataset.__len__()}")
print(f"img : {val_dataset.__getitem__(index)[0].size()}")
print(f"label : {val_dataset.__getitem__(index)[1]}")
print(f"image name : {val_dataset.__getitem__(index)[2]}")

print("\n【test dataset】")
print(f"img num : {test_dataset.__len__()}")
print(f"img : {test_dataset.__getitem__(index)[0].size()}")
print(f"label : {test_dataset.__getitem__(index)[1]}")
print(f"image name : {test_dataset.__getitem__(index)[2]}")

## Build Dataloader Object

In [ ]:
batch_size = 128

# Create DataLoader
train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# to Dictionary
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader, "test": test_dataloader}

# Operation check
#batch_iterator = iter(dataloaders_dict["train"])
#inputs, labels = next(batch_iterator)
#print(inputs.size())  # torch.Size([3, 3, 224, 224]) : [batch_size, Channel, H, W]
#print(labels)

## Network model

In [ ]:
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in net.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

* ## VGG16

* ### Load Model

In [ ]:
# ! ls /kaggle/input/pytorch-pretrained-models

In [ ]:
if SELECT_MODEL == 'VGG16':
    # Load the learned VGG-16 model.

    # Create an instance of the VGG-16 model
    use_pretrained = False
    net = models.vgg16(pretrained=use_pretrained)

    #save_path = "/kaggle/working/vgg16_pretrained.h"
    #torch.save(net.state_dict(), save_path)

* ### Change Model's Last Layer

In [ ]:
if SELECT_MODEL == 'VGG16':

    # Replace the output unit of the last output layer of the VGG-16 model.
    # out_features 1000 to 12
    net.classifier[6] = nn.Linear(in_features=4096, out_features=12)

    # Newly created modules have require_grad=True by default
#     num_features = net.classifier[6].in_features

#     print(list(net.classifier.children()))
#     features = list(net.classifier.children())[:-1] # Remove last layer
#     features.extend([
#         nn.Linear(num_features, 2048),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.5, inplace=False),
#         nn.Linear(2048, 512),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.5, inplace=False),
#         nn.Linear(512, 64),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.5, inplace=False),
#         nn.Linear(64, 12)
#     ])
#     net.classifier = nn.Sequential(*features) # Replace the model classifier
#     print(net)

    # Set to training mode.
    net.train()

* ### Optimizer

In [ ]:
if SELECT_MODEL == 'VGG16':
    for name, param in net.named_parameters():
        print(name)

In [ ]:
if SELECT_MODEL == 'VGG16':
    # Store the parameters to be learned by finetuning in the variable params_to_update.
    params_to_update_1 = []
    params_to_update_2 = []
    params_to_update_3 = []
    params_to_update_4 = []

    # Specify the parameter name of the layer to be trained.

    update_param_names_1 = ["features.24.weight", "features.24.bias", "features.26.weight", "features.26.bias", "features.28.weight", "features.28.bias"]
    update_param_names_2 = ["classifier.0.weight", "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
    update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]
    update_param_names_4 = [
        "features.10.weight", "features.10.bias", 
        "features.12.weight", "features.12.bias",
        "features.14.weight", "features.14.bias", 
        "features.17.weight", "features.17.bias",
        "features.19.weight", "features.19.bias",
        "features.21.weight", "features.21.bias"
    ]

    for name, param in net.named_parameters():
        if name in update_param_names_1:
            param.requires_grad = False
            params_to_update_1.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_1 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_2:
            param.requires_grad = False
            params_to_update_2.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_2 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_3:
            param.requires_grad = False
            params_to_update_3.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_3 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_4:
            param.requires_grad = True
            params_to_update_4.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_4 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        else:
            param.requires_grad = False
            print(f"Parameters not to be learned :  {name}")

In [ ]:
if SELECT_MODEL == 'VGG16':
    # Set Optimizer
    optimizer = optim.SGD([
        {"params": params_to_update_1, "lr": 1e-4},
        {"params": params_to_update_2, "lr": 1e-4},
        {"params": params_to_update_3, "lr": 1e-4}
    ], momentum=0.9)

* ## AlexNet

* ### Load Model

In [ ]:
if SELECT_MODEL == 'AlexNet':

    load_path_alexNet = "/kaggle/input/pytorch-pretrained-models/alexnet-owt-4df8aa71.pth"

    net = models.alexnet(pretrained=False)

    # Replace the output unit of the last output layer of the VGG-16 model.
    # out_features 1000 to 12
    net.classifier[6] = nn.Linear(in_features=4096, out_features=12)

    # Set to training mode.
    net.train()

* ### Optimizer

In [ ]:
if SELECT_MODEL == 'AlexNet':
    for name, param in net.named_parameters():
        print(name)

In [ ]:
if SELECT_MODEL == 'AlexNet':
    # Store the parameters to be learned by finetuning in the variable params_to_update.
    params_to_update_1 = []
    params_to_update_2 = []
    params_to_update_3 = []

    # Specify the parameter name of the layer to be trained.
    update_param_names_1 = ["features.0.weight", "features.0.bias","features.3.weight", "features.3.bias", "features.6.weight", "features.6.bias", "features.8.weight", "features.8.bias", "features.10.weight", "features.10.bias"]
    update_param_names_2 = ["classifier.1.weight", "classifier.1.bias", "classifier.4.weight", "classifier.4.bias"]
    update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]


    for name, param in net.named_parameters():
        if name in update_param_names_1:
            param.requires_grad = True
            params_to_update_1.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_1 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_2:
            param.requires_grad = True
            params_to_update_2.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_2 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_3:
            param.requires_grad = True
            params_to_update_3.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_3 : {name}")
            else:
                print(f"Parameters not to be learned :  {name}")
        else:
            param.requires_grad = False
            print(f"Parameters not to be learned :  {name}")

In [ ]:
if SELECT_MODEL == 'AlexNet':
    # Set Optimizer
    optimizer = optim.SGD([
        {"params": params_to_update_1, "lr": 5e-4},
        {"params": params_to_update_2, "lr": 5e-4},
        {"params": params_to_update_3, "lr": 1e-3}
    ], momentum=0.9)

* ## DenseNet

* ### Load Pretrained Model

In [ ]:
if SELECT_MODEL == 'DenseNet':

#     load_path = "../input/pytorch-pretrained-models/densenet161-347e6b360.pth"

    net = models.densenet161(pretrained= False)

#     if torch.cuda.is_available():
#         load_weights = torch.load(load_path)
#         net.load_state_dict(load_weights)
#     else:
#         load_weights = torch.load(load_path, map_location={"cuda:0": "cpu"})
#         net.load_state_dict(load_weights)

* ### Modify Model Fit Problem

In [ ]:
if SELECT_MODEL == 'DenseNet':

    # Replace the output unit of the last output layer of the VGG-16 model.
    # out_features 1000 to 12

    prev_out_feature = net.classifier.in_features
    
#     new_last_layer = [
#         nn.Linear(prev_out_feature, 2208),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.5, inplace=False),
        
#         nn.Linear(2208, 1104),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.5, inplace=False),
        
#         nn.Linear(1104, 552),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.2, inplace=False),
        
#         nn.Linear(552, 138),
#         nn.ReLU(inplace = True),
#         nn.Dropout(p=0.2, inplace=False),
        
#         nn.Linear(138, 12),
#     ]
    new_last_layer = [
        nn.Linear(prev_out_feature, 12)
    ]
    
    net.classifier = nn.Sequential(*new_last_layer)

In [ ]:
if SELECT_MODEL == 'DenseNet':
    net.classifier.train()

In [ ]:
if SELECT_MODEL == 'DenseNet':
#     a = 'features.denseblock3'
#     print([f'{a}.{name}' for name, _ in net_densenet.features.denseblock3.named_parameters()])
#     for name, param in net_densenet.named_parameters():
#         print(name)
    pass

* ### Optimizer

In [ ]:
if SELECT_MODEL == 'DenseNet':
    # Store the parameters to be learned by finetuning in the variable params_to_update.
    params_to_update_1 = []
    params_to_update_2 = []
    params_to_update_3 = []

    # Specify the parameter name of the layer to be trained.
    update_param_names_1 = [f'features.denseblock4.{name}' for name, _ in net.features.denseblock4.named_parameters()]
    update_param_names_2 = [f'features.denseblock3.{name}' for name, _ in net.features.denseblock3.named_parameters()]
    update_param_names_3 = [f'classifier.{name}' for name, _ in net.classifier.named_parameters()]


    for name, param in net.named_parameters():
        if name in update_param_names_1:
            param.requires_grad = False
            params_to_update_1.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_1 : {name}")
#             else:
#                 print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_2:
            param.requires_grad = False
            params_to_update_2.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_2 : {name}")
#             else:
#                 print(f"Parameters not to be learned :  {name}")
        elif name in update_param_names_3:
            param.requires_grad = True
            params_to_update_3.append(param)
            if param.requires_grad:
                print(f"Store in params_to_update_3 : {name}")
#             else:
#                 print(f"Parameters not to be learned :  {name}")
        else:
            param.requires_grad = False
            if param.requires_grad:
                print(f"Store in params_to_update_3 : {name}")
#             print(f"Parameters not to be learned :  {name}")

In [ ]:
if SELECT_MODEL == 'DenseNet':
    # Set Optimizer
    optimizer = optim.Adam([
        {"params": params_to_update_1, "lr": 5e-4},
        {"params": params_to_update_2, "lr": 1e-4},
        {"params": params_to_update_3, "lr": 1e-4}
    ], lr = 1e-4)

## Load Pretrained Params

In [ ]:
print(PRETRAIN_PATH)
if LOAD_PRETRAIN:
    if torch.cuda.is_available():
        load_weights = torch.load(PRETRAIN_PATH)
        net.load_state_dict(load_weights)
    else:
        load_weights = torch.load(PRETRAIN_PATH, map_location={"cuda:0": "cpu"})
        net.load_state_dict(load_weights)
    print('loaded pretrained')

## Define Loss Function

In [ ]:
criterion = nn.CrossEntropyLoss()

## Function for model training

In [ ]:
from sklearn.metrics import f1_score, accuracy_score

def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):
    """
    Function for training the model.
    
    Parameters
    ----------
    net: object
    dataloaders_dict: dictionary
    criterion: object
    optimizer: object
    num_epochs: int
    """
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Devices to be used : {device}")
    net.to(device)
    torch.backends.cudnn.benchmark = True
    # loop for epoch
    train_acc_list = []
    train_loss_list = []
    train_f1_score_list = []
    val_acc_list = []
    val_loss_list = []
    val_f1_score_list = []
    for epoch in range(num_epochs):
        print(f"Epoch {epoch+1} / {num_epochs}")
        print("-------------------------------")
        for phase in ["train", "val"]:
            if phase == "train":
                net.train()
            else:
                net.eval()
            epoch_loss = 0.0
            epoch_corrects = 0
            epoch_f1 = 0
            #if (epoch == 0) and (phase == "train"):
                #continue
            epoch_num = 0
            for inputs, labels, _ in tqdm(dataloaders_dict[phase]):
                epoch_num += 1
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == "train"):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    if phase == "train":
                        loss.backward()
                        optimizer.step()
                    _loss = loss.item() * inputs.size(0)
                    _correct = torch.sum(preds == labels.data)
                    _acc = 1.0 * _correct / len(inputs)
                    _f1_score = f1_score(preds.cpu().data.numpy(), labels.cpu().data.numpy(), average='weighted')
                    print(f"Tracking - {phase} Loss: {_loss:.4f} Acc: {_acc:.4f}  F1-Score: {_f1_score:4f}")
                    epoch_loss += _loss
                    epoch_corrects += _correct
                    epoch_f1 += _f1_score
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)
            epoch_f1 = epoch_f1 / epoch_num
            if phase == "train":
                train_acc_list.append(epoch_acc)
                train_loss_list.append(epoch_loss)
                train_f1_score_list.append(epoch_f1)
            else:
                net.eval()
                val_acc_list.append(epoch_acc)
                val_loss_list.append(epoch_loss)
                val_f1_score_list.append(epoch_f1)
            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1-Score: {epoch_f1:4f}")
    return train_acc_list, train_loss_list, train_f1_score_list, val_acc_list, val_loss_list, val_f1_score_list

# Start training 

In [ ]:
torch.cuda.is_available()

### Start Training

In [ ]:
if not IS_SUBMIT:
    num_epochs = 10
    train_acc_list, train_loss_list, train_f1_score_list, val_acc_list, val_loss_list, val_f1_score_list = train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)
    # Save Model Weight
    save_path = f"./{SAVE_WEIGHT_FILENAME}.h"
    torch.save(net.state_dict(), save_path)

In [ ]:
if not IS_SUBMIT:
    plt.plot(train_acc_list, label='Train acc')
    plt.plot(val_acc_list, label='Val acc')
    plt.legend()

In [ ]:
if not IS_SUBMIT:
    plt.plot(train_loss_list, label='Train loss')
    plt.plot(val_loss_list, label='Val loss')
    plt.legend()

In [ ]:
if not IS_SUBMIT:
    plt.plot(train_f1_score_list, label='Train f1')
    plt.plot(val_f1_score_list, label='Val f1')
    plt.legend()

## load weights

In [ ]:
# load_path = "/kaggle/working/vgg16_second_fine_tuning_v1.h"
if IS_SUBMIT:
    if torch.cuda.is_available():
        load_weights = torch.load(PRETRAIN_PATH)
        net.load_state_dict(load_weights)
    else:
        load_weights = torch.load(PRETRAIN_PATH, map_location={"cuda:0": "cpu"})
        net.load_state_dict(load_weights)

In [ ]:
net

## inference

In [ ]:
#batch_iterator = iter(dataloaders_dict["val"])
#inputs, labels, image_name = next(batch_iterator)
#print(inputs.size())  # torch.Size([3, 3, 224, 224]) : [batch_size, Channel, H, W]
#print(labels)

In [ ]:
class PlantPredictor():
    """
    Class for predicting labels from output results
    
    Attributes
    ----------
    df_labels_idx: DataFrame
        DataFrame that associates INDEX with a label name
    """
    
    def __init__(self, net, df_labels_idx, dataloaders_dict):
        self.net = net
        self.df_labels_idx = df_labels_idx
        self.dataloaders_dict = dataloaders_dict
        self.df_submit = pd.DataFrame()
        
    
    def __predict_max(self, out):
        """
        Get the label name with the highest probability.
        
        Parameters
        ----------
        predicted_label_name: str
            Name of the label with the highest prediction probability
        """
        maxid = np.argmax(out.detach().numpy(), axis=1)
        df_predicted_label_name = self.df_labels_idx.iloc[maxid]
        
        return df_predicted_label_name
    
    def inference(self):
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        print(f"Devices to be used : {device}")
        df_pred_list = []
        for inputs, _, image_name in tqdm(self.dataloaders_dict['test']):
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
            self.net.to(device)
            inputs = inputs.to(device)
            out = self.net(inputs)
            device = torch.device("cpu")
            out = out.to(device)
            df_pred = self.__predict_max(out).reset_index(drop=True)
            df_pred["image"] = image_name
            df_pred_list.append(df_pred)
            
        self.df_submit = pd.concat(df_pred_list, axis=0)
        self.df_submit = self.df_submit[["image", "labels"]].reset_index(drop=True)
            
        

In [ ]:
predictor = PlantPredictor(net, df_labels_idx, dataloaders_dict)
predictor.inference()
#df_pred = predictor.predict_max(out)

#df_sub.labels = df_pred.labels.reset_index(drop=True)
#display(df_pred)
#display(df_sub)

In [ ]:
df_submit = predictor.df_submit.copy()

## Submit

In [ ]:
df_submit.to_csv("/kaggle/working/submission.csv", index=False)